In [ ]:
# pip install -U textblob

In [1]:
# For EDA now, will add more libraries as we progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from textblob import TextBlob

# Set nice style for plots
sns.set_theme(style='darkgrid')
sns.dark_palette("#69d", reverse=True, as_cmap=True)
sns.set_context("paper")



# 1. Read Data

Our data was taken from this database of [Google Local Data 2021](https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/#subsets). We previously used a smaller dataset of Google Restaurant reviews available [here](https://www.kaggle.com/datasets/denizbilginn/google-maps-restaurant-reviews), with only 1100 reviews, but as per our TF's recommendation, we will be using this larger dataset with over 10m reviews for Massachusetts alone. To speed up our EDA, we will be using a sample of 100,000 reviews.

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Our main dataset, stored in the `df` variable, contains 100,000 reviews. Our columns of interest are `rating`, which is the rating given by the user, `text`, which is the review text, and `gmap_id`, which is the Google Maps ID of the business. 

We also have a metadata dataset, stored in the `df_meta` variable, which contains information about each business in Massachusetts. The columns of interest are `gmap_id`, allowing us to join this dataset with the main one, `name`, which is the name of the business, and `description`, which is a brief description of the business. There is also a variable `category` that organizes the businesses into sectors such as non-profits, gyms, restaurants, etc. 

In [2]:
# Load the data
with open('data/review-Massachusetts.json', 'r') as f:
    data = f.readlines()

# Convert to dataframe
data_json_str = "[" + ','.join(data) + "]"
df = pd.read_json(StringIO(data_json_str))

In [3]:
# Examine df
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10447007 entries, 0 to 10447006
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   float64
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 637.6+ MB
None
        user_id                 name           time  rating  \
0  1.089901e+20         Sherri Mayne  1559246854759     5.0   
1  1.004254e+20       Adam Goodspeed  1557586283214     5.0   
2  1.181979e+20  Christopher Sheehan  1568756947956     4.0   
3  1.063415e+20       Katie Sullivan  1627234799114     5.0   
4  1.024727e+20   Victoria Henderson  1619237055452     5.0   

                                                text  pics  resp  \
0  I love the people that live there, they ate th...  None  None   
1  Stop parking in the residents spots for your d...  None  None   
2             

In [4]:
# Also load the whole meta-Massachusetts.json file. This contains metadata about the businesses, 
# since the reviews only contain the business id.
with open('data/meta-Massachusetts.json', 'r') as f:
    data_meta = f.readlines()

data_meta_str = "[" + ','.join(data_meta) + "]"
df_meta = pd.read_json(StringIO(data_meta_str))

# Filter dataframe so that it only contains restaurants
df_meta = df_meta[df_meta['category'].apply(lambda x: isinstance(x, list) and any('restaurant' in category.lower() for category in x) if x is not None else False)]

In [5]:
# Examine df
print(df_meta.info())
print(df_meta.head())

<class 'pandas.core.frame.DataFrame'>
Index: 16079 entries, 14 to 92514
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              16079 non-null  object 
 1   address           16055 non-null  object 
 2   gmap_id           16079 non-null  object 
 3   description       9341 non-null   object 
 4   latitude          16079 non-null  float64
 5   longitude         16079 non-null  float64
 6   category          16079 non-null  object 
 7   avg_rating        16079 non-null  float64
 8   num_of_reviews    16079 non-null  int64  
 9   price             11987 non-null  object 
 10  hours             15127 non-null  object 
 11  MISC              15994 non-null  object 
 12  state             11728 non-null  object 
 13  relative_results  15090 non-null  object 
 14  url               16079 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 2.0+ MB
None
                            

In [6]:
# Merging the dataframes on 'gmap_id'
# 'inner' will only include rows that have matching 'gmap_id' in both dataframes
df_combined = pd.merge(df, df_meta, on='gmap_id', how='inner')

In [7]:
# Examine the combined dataframe
print(df_combined.info())
print(df_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182567 entries, 0 to 4182566
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           float64
 1   name_x            object 
 2   time              int64  
 3   rating            float64
 4   text              object 
 5   pics              object 
 6   resp              object 
 7   gmap_id           object 
 8   name_y            object 
 9   address           object 
 10  description       object 
 11  latitude          float64
 12  longitude         float64
 13  category          object 
 14  avg_rating        float64
 15  num_of_reviews    int64  
 16  price             object 
 17  hours             object 
 18  MISC              object 
 19  state             object 
 20  relative_results  object 
 21  url               object 
dtypes: float64(5), int64(2), object(15)
memory usage: 702.0+ MB
None
        user_id      name_x           time  rating  \
0  1.053246e+20    

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

We merged the main dataset and our meta dataset based on `gmap_id` and filtered out the ones of restaurants.

We will clean up the dataframe for our purpose. From below we see that we can probably get rid of these columns: `name_x` (since we don't care about the name of the reviewer, having `user_id` is enough to identify them; `time` (for now we don't care about when the reviewers wrote the reviews); `pics` (we don't care about pictures first, but might use it in the future if time permits); `resp` (most of them are `None`); `gmap_id`(we can identify restaurants based on their names which is easier to understand); `description` (most of them are `None`); `relative_results`; `url`)

We see that there are duplicates in our merged dataset. We will drop the duplicates after we clean up the dataframe to only keep necessary columns.

In [8]:
df_combined.head()

,user_id,name_x,time,rating,text,pics,resp,gmap_id,name_y,address,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,1.053246e+20,Jessica,1515819902193,4.0,What a great experience. I tried the chicken t...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
1,1.053246e+20,Jessica,1515819902193,4.0,What a great experience. I tried the chicken t...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
2,1.115933e+20,Jonah Ford,1526075323967,5.0,I've probably driven past this place a million...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
3,1.115933e+20,Jonah Ford,1526075323967,5.0,I've probably driven past this place a million...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
4,1.045125e+20,Zak Bug,1497983845749,5.0,The food was fantastic. The staff was very fr...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...


In [10]:
# Drop duplicates based on `user_id` and `gmap_id' to ensure unique reviews per user per restaurant
df_cleaned = df_combined.drop_duplicates(subset=['user_id', 'gmap_id'])

df_cleaned.head()

,user_id,name_x,time,rating,text,pics,resp,gmap_id,name_y,address,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,1.053246e+20,Jessica,1515819902193,4.0,What a great experience. I tried the chicken t...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
2,1.115933e+20,Jonah Ford,1526075323967,5.0,I've probably driven past this place a million...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
4,1.045125e+20,Zak Bug,1497983845749,5.0,The food was fantastic. The staff was very fr...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
6,1.085975e+20,Joey Taraborrelli,1529934471266,5.0,"The owner, T, is a wonderful man. Super friend...",None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...
8,1.182263e+20,John Mahoney,1491413822507,5.0,Just stopped in for a couple of grilled chicke...,None,None,0x89e3169821e62d4d:0x14ff0683c1ebca0e,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",...,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...


In [11]:
columns_to_drop = ['name_x', 'time', 'pics', 'resp', 'gmap_id', 'description', 'relative_results', 'url']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

In [12]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4135196 entries, 0 to 4182566
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         float64
 1   rating          float64
 2   text            object 
 3   name_y          object 
 4   address         object 
 5   latitude        float64
 6   longitude       float64
 7   category        object 
 8   avg_rating      float64
 9   num_of_reviews  int64  
 10  price           object 
 11  hours           object 
 12  MISC            object 
 13  state           object 
dtypes: float64(5), int64(1), object(8)
memory usage: 473.2+ MB


<div style="background-color:#3F7FBF; color:white; padding:10px"> 

The `MISC` column contains dictionaries with useful information. We decided to further process this column and extract the attributes out as new columns of our dataframe.

In [13]:
print(df_cleaned.loc[0, 'MISC'])

{'Service options': ['Takeout', 'Delivery'], 'Accessibility': ['Wheelchair accessible entrance'], 'Offerings': ['Comfort food', 'Late-night food', 'Vegetarian options'], 'Dining options': ['Lunch', 'Dinner', 'Dessert'], 'Payments': ['Debit cards']}


In [14]:
# First, ensure all entries in 'MISC' are dictionaries; replace None with empty dictionaries
df_cleaned['MISC'] = df_cleaned['MISC'].apply(lambda x: x if isinstance(x, dict) else {})

# Convert the 'MISC' column to a DataFrame where each key in the dictionary becomes a column
misc_expanded = pd.DataFrame(df_cleaned['MISC'].tolist())

In [15]:
# Join the expanded 'MISC' DataFrame with the original 'df_cleaned', excluding the 'MISC' column
df_expanded = pd.concat([df_cleaned.drop(columns=['MISC']), misc_expanded], axis=1)

In [16]:
df_expanded.head()

,user_id,rating,text,name_y,address,latitude,longitude,category,avg_rating,num_of_reviews,...,Payments,Highlights,Popular for,Amenities,Atmosphere,Health & safety,Crowd,Planning,From the business,Health and safety
0,1.053246e+20,4.0,What a great experience. I tried the chicken t...,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,...,[Debit cards],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.115933e+20,5.0,I've probably driven past this place a million...,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,...,[Debit cards],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.045125e+20,5.0,The food was fantastic. The staff was very fr...,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,...,[Debit cards],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.085975e+20,5.0,"The owner, T, is a wonderful man. Super friend...",Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,...,[Debit cards],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.182263e+20,5.0,Just stopped in for a couple of grilled chicke...,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,...,[Debit cards],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4182567 entries, 0 to 3957456
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            float64
 1   rating             float64
 2   text               object 
 3   name_y             object 
 4   address            object 
 5   latitude           float64
 6   longitude          float64
 7   category           object 
 8   avg_rating         float64
 9   num_of_reviews     float64
 10  price              object 
 11  hours              object 
 12  state              object 
 13  Service options    object 
 14  Accessibility      object 
 15  Offerings          object 
 16  Dining options     object 
 17  Payments           object 
 18  Highlights         object 
 19  Popular for        object 
 20  Amenities          object 
 21  Atmosphere         object 
 22  Health & safety    object 
 23  Crowd              object 
 24  Planning           object 
 25  From the business  obje

In [18]:
columns_of_interest = [
    "Service options", "Accessibility", "Offerings", "Dining options",
    "Payments", "Highlights", "Popular for", "Amenities",
    "Atmosphere", "Health & safety", "Crowd", "Planning",
    "From the business", "Health and safety"
]

# Initialize a dictionary to hold the unique values for each column
unique_values = {}

# Iterate over each column of interest
for column in columns_of_interest:
    # Extract the column's data
    column_data = df_expanded[column]
    
    # Since the data might contain lists, we need to flatten these into a single list before finding unique values
    # We'll use a set to automatically keep only unique items
    flattened_set = set()
    for item in column_data.dropna():  # Drop NA values to avoid errors
        if isinstance(item, list):  # Check if the item is a list
            flattened_set.update(item)  # Add all items in the list to the set
        else:
            flattened_set.add(item)  # Add the item itself if it's not a list
    
    # Store the unique values for this column in our dictionary
    unique_values[column] = list(flattened_set)  # Convert the set back to a list for readability

# Or, to print the unique values for all columns of interest:
for column, values in unique_values.items():
    print(f"{column}: {values}\n")

Service options: ['In-store pick-up', 'No-contact delivery', 'In-store shopping', 'Outdoor seating', 'Delivery', 'Takeout', 'Dine-in', 'Same-day delivery', 'Takeaway', 'Drive-through', 'In-store pickup', 'Online appointments', 'Curbside pickup']

Accessibility: ['Wheelchair accessible seating', 'Wheelchair accessible restroom', 'Assisted listening devices', 'Wheelchair accessible elevator', 'Wheelchair-accessible entrance', 'Wheelchair accessible entrance', 'Wheelchair-accessible toilet', 'Wheelchair-accessible lift', 'Wheelchair-accessible car park', 'Wheelchair accessible parking lot', 'Wheelchair rental', 'Wheelchair-accessible seating']

Offerings: ['Service guarantee', 'Happy hour food', 'Organic products', 'Cocktails', 'Salad bar', 'Spirits', 'Happy hour drinks', 'Quick bite', 'Ethanol-free gas', 'Hard liquor', 'Comfort food', 'Organic dishes', 'Healthy options', 'Full service gas', 'Prepared foods', 'Alcohol', 'Vegetarian options', 'Car wash', 'Happy-hour drinks', 'Wine', 'Coffe

In [19]:
# Counting NaN values in each column of df_expanded
nan_counts = df_expanded.isna().sum()

# Displaying the count of NaN values per column
print(nan_counts)

user_id                47393
rating                 47393
text                 1939737
name_y                 47371
address                48769
latitude               47371
longitude              47371
category               47371
avg_rating             47371
num_of_reviews         47371
price                 432281
hours                 156138
state                1891583
Service options       148804
Accessibility         412830
Offerings             359532
Dining options        688883
Payments             1605942
Highlights           1359587
Popular for           710651
Amenities             444311
Atmosphere            629354
Health & safety      1675107
Crowd                 666088
Planning             2167329
From the business    4008956
Health and safety    4115234
dtype: int64


<div style="background-color:#3F7FBF; color:white; padding:10px"> 
Since we will focus on reviews, we decided to drop rows with `text` (which is the review column) empty. We will also drop the rows with `rating` empty.

In [20]:
# Drop rows where the 'text' column is NaN
df_expanded_clean = df_expanded.dropna(subset=['text', 'rating'])
df_expanded_clean.isna().sum()

user_id                    0
rating                     0
text                       0
name_y                     0
address                  876
latitude                   0
longitude                  0
category                   0
avg_rating                 0
num_of_reviews             0
price                 228738
hours                  60330
state                 972148
Service options        79738
Accessibility         225759
Offerings             193532
Dining options        373768
Payments              865832
Highlights            733863
Popular for           381437
Amenities             239351
Atmosphere            338583
Health & safety       901797
Crowd                 361852
Planning             1165285
From the business    2150732
Health and safety    2207346
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 
    
Let's further inspect these columns and decide either to drop them or process them.

`Planning`, `From the business`, and `Health and safety` simply just have too many missing values, so we will drop them.

We decided to drop `address` column since we will be able to infer the address based on names of the restaurants and the latitude and longitude.

`state` describes the current states of the restraurants when the dataset authors scraped it. This is not that helpful for our analysis so we will drop this column too.

There are some other variables which are not important for our analysis for now. We will drop them as well and start thinking about how to process the rest (e.g. one-hot encode).

In [21]:
df_expanded_clean['state'].unique()

array(['Permanently closed', None,
       'Closes soon ⋅ 8:30PM ⋅ Opens 11AM Thu', 'Closed ⋅ Opens 7AM Thu',
       'Open ⋅ Closes 10PM', 'Closes soon ⋅ 4PM ⋅ Opens 8AM Thu',
       'Open ⋅ Closes 8PM', 'Open ⋅ Closes 5PM', 'Closed ⋅ Opens 11AM',
       'Closed ⋅ Opens 11:30AM', 'Closed ⋅ Opens 5PM Thu',
       'Open ⋅ Closes 2AM', 'Closed ⋅ Opens 11AM Wed',
       'Closed ⋅ Opens 8AM Wed', 'Closed ⋅ Opens 8:30AM Wed',
       'Closed ⋅ Opens 6AM Wed', 'Closed ⋅ Opens 7:30AM Wed',
       'Open ⋅ Closes 8AM Wed', 'Opens soon ⋅ 9AM',
       'Closed ⋅ Opens 8:30AM', 'Closed ⋅ Opens 4PM Thu',
       'Closed ⋅ Opens 5AM', 'Closed ⋅ Opens 7:15AM',
       'Closes soon ⋅ 9PM ⋅ Opens 10AM Tue', 'Open 24 hours',
       'Open ⋅ Closes 9PM', 'Closed ⋅ Opens 4PM', 'Closed ⋅ Opens 7AM',
       'Closed ⋅ Opens 7:30AM Mon', 'Closed ⋅ Opens 5:30AM Mon',
       'Closed ⋅ Opens 3PM Tue', 'Closed ⋅ Opens 8:30AM Mon',
       'Closed ⋅ Opens 11AM Mon', 'Closed ⋅ Opens 12PM',
       'Closed ⋅ Opens 9AM', 'Clo

In [22]:
df_expanded_clean['price'].unique()

array(['$$', None, '$', '$$$', '$$$$', '₩₩', '₩', '₩₩₩'], dtype=object)

In [34]:
columns_to_drop = ['state', 'address', 'hours', 'Service options', 'Accessibility', 'Offerings', 'Highlights', 'From the business', 'Health and safety', 'Health & safety', 'Planning', 'Payments', 'Amenities']
df_processed = df_expanded_clean.drop(columns=columns_to_drop)

In [35]:
df_processed.isna().sum()

user_id                0
rating                 0
text                   0
name_y                 0
latitude               0
longitude              0
category               0
avg_rating             0
num_of_reviews         0
price             228738
Dining options    373768
Popular for       381437
Atmosphere        338583
Crowd             361852
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Since our dataset is large enough, for the remaining missing values, we decided to just simple drop the rows with missing `Popular for`, `Dining options`, and `price`.

In [36]:
df_processed2 = df_processed.dropna(subset=['Popular for', 'Dining options', 'price'])
df_processed2.isna().sum()

user_id               0
rating                0
text                  0
name_y                0
latitude              0
longitude             0
category              0
avg_rating            0
num_of_reviews        0
price                 0
Dining options        0
Popular for           0
Atmosphere         1279
Crowd             39439
dtype: int64

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

We can encode the missing values in `Atmosphere` as 'unknown', and the ones in `Crowd` as 'any'.

We will also need to process the representating of `price`. We decided to encode the `$` symbols as numbers, so `$` = 1, `$$` = 2, an so on.


In [37]:
# Replace NA values in 'Atmosphere' column with 'unknown'
df_processed2.loc[:, 'Atmosphere'] = df_processed2['Atmosphere'].fillna('unknown')

# Replace NA values in 'Crowd' column with 'any'
df_processed2.loc[:, 'Crowd'] = df_processed2['Crowd'].fillna('any')

In [38]:
df_processed2.isna().sum()

user_id           0
rating            0
text              0
name_y            0
latitude          0
longitude         0
category          0
avg_rating        0
num_of_reviews    0
price             0
Dining options    0
Popular for       0
Atmosphere        0
Crowd             0
dtype: int64

In [39]:
def encode_price(price):
    if pd.isna(price):
        return None  # Use np.nan 
    price = price.replace('₩', '$')  # Normalize '₩' to '$'
    return len(price)  # The number of '$' symbols corresponds to the price level


In [40]:
df_processed2.loc[:, 'price'] = df_processed2['price'].apply(encode_price)

In [41]:
df_processed2.head()

,user_id,rating,text,name_y,latitude,longitude,category,avg_rating,num_of_reviews,price,Dining options,Popular for,Atmosphere,Crowd
48,1.085140e+20,5.0,Amazing food and T is an amazing man.,Three Star Pizza,42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,2,"[Breakfast, Lunch, Dinner, Dessert]","[Breakfast, Lunch, Dinner, Solo dining]","[Casual, Cozy]",any
50,1.002992e+20,3.0,I didn't actually try it but just felt like gi...,Three Star Pizza,42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,2,"[Breakfast, Lunch, Dinner, Dessert]","[Breakfast, Lunch, Dinner, Solo dining]","[Casual, Cozy]",any
52,1.176323e+20,5.0,I am a huge fan of their sandwiches! Made to o...,Three Star Pizza,42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,2,"[Breakfast, Lunch, Dinner, Dessert]","[Breakfast, Lunch, Dinner, Solo dining]","[Casual, Cozy]",any
54,1.006278e+20,2.0,There's a reason why it's not called 5 star pi...,Three Star Pizza,42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,2,"[Breakfast, Lunch, Dinner, Dessert]","[Breakfast, Lunch, Dinner, Solo dining]","[Casual, Cozy]",any
56,1.138433e+20,4.0,Drunk and starving at 1am? They're open!,Three Star Pizza,42.559072,-70.881542,"[Pizza restaurant, Italian restaurant, Deliver...",3.9,48.0,2,"[Breakfast, Lunch, Dinner, Dessert]","[Breakfast, Lunch, Dinner, Solo dining]","[Casual, Cozy]",any


In [42]:
# df_processed2.to_csv('data/cleaned_data.csv')

In [43]:
df_final = pd.read_csv('data/cleaned_data.csv')

# 2. EDA

## Exploring the distribution of ratings

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* Ratings are on a 1 to 5 scale.
* The majority of reviews are positive, with 77870 reviews rated 5.
* However, we note that the second most common rating is 1 at 9435 reviews. This suggests that people leave reviews either when they have a highly good or highly bad experience at a restaurant. 

In [ ]:
# Distribution of ratings: To understand the overall sentiment towards the businesses.

plt.hist(df['rating'], bins = 5, edgecolor = 'black')
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
# show exact counts
for i in range(1, 6):
    plt.text(i + 0.5 if i == 1 else i + 0.3 if i == 2 else i + 0.1 if i == 3 else i if i == 4 else i - 0.2, len(df[df['rating'] == i]), str(len(df[df['rating'] == i])), ha='right', va='bottom')
# change x ticks to read 1, 2, 3, 4, 5
plt.xticks(np.arange(1, 6, 1))

plt.show()


## Exploring review counts per local business

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* Most businesses have 10-20 reviews, with the number of businesses decreasing at a decreasing rate as the number of reviews increases.
* The mean number of reviews is 20.5 with a standard deviation of 40.53.
* This suggests that a small number of businesses might be overrepresented in our dataset, especially the one outlying businesses with 1292 reviews.

In [ ]:
# Count of reviews per business - To see which businesses have been reviewed the most.

# describe the count of reviews per business
print("Summary statistics for count of reviews per business:")
print(df['gmap_id'].value_counts().describe())

df['gmap_id'].value_counts().plot(kind='hist', bins=100, edgecolor='black')
plt.title('Count of Reviews per Business')
plt.xlabel('Number of Reviews')
plt.ylabel('Count of Businesses')
plt.show()




## Exploring review lengths (in terms of number of characters)

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We see that review lengths are distributed similarly to business ratings, with a large number of reviews having under 100 characters, and with the number of reviews decreasing at a decreasing rate as the review length increases.
* The mean review length is 167.47, but with a large standard deviation of 273.39.

In [ ]:
# Review length analysis: To see the distribution of the length of the review texts.

df['review_length'] = df['text'].apply(lambda x: len(x) if x is not None else 0)

# describe the review length
print("Summary statistics for review length:")
print(df['review_length'].describe())

plt.hist(df['review_length'], bins=100, edgecolor='black')
plt.title('Distribution of Review Length')
plt.xlabel('Review Length')
plt.ylabel('Count')
plt.show()



## Exploring correlation between review length and rating

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We analyzed the correlation between the length of the review text and the rating. The correlation coefficient is around -0.156, which suggests that there is only a weak negative correlation between the length of the review and the rating.
* This result is consistent with our intuition, as the length of a review does not necessarily indicate its quality or sentiment.
* However, perhaps some angrier customers might leave longer reviews, which could explain the slight negative correlation. 
* We also see that reviews with a rating of 3 have a relatively lower review length, which supports the overlying notion that people write more/longer reviews when they feel strongly about a restaurant. 

In [ ]:
# Correlation between review length and rating: To see if there is a correlation between the length of the review and the rating given.

# plot the correlation between review length and rating
sns.scatterplot(x='rating', y='review_length', data=df)
plt.title('Correlation between Review Length and Rating')
plt.xlabel('Rating')
plt.ylabel('Review Length')
plt.xticks(np.arange(1, 6, 1))
plt.show()

correlation_matrix = df[['rating', 'review_length']].corr()
print("Correlation matrix:")
print(correlation_matrix)

## Number of unique authors

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We see that we have 77593 unique authors for our 100,000 reviews, which suggests that most authors have only left one review.
* This is confirmed by the mean number of reviews per author, which is 1.3 with a standard deviation of 0.88.
* In fact, we see through the summary statistics that over 75% of authors have only left 1 review. The most number of reviews left by one author is 62. 
* The shape of the distribution is once again similar to the previous two, with the number of authors decreasing at a decreasing rate as the number of reviews per author increases.

In [ ]:
# How many unique authors?

print("Number of unique authors: ", df['name'].nunique())

# describe the count of ratings per author
print("Summary statistics for number of ratings per author:")
print(df['name'].value_counts().describe())

df['name'].value_counts().plot(kind='hist', bins=100, edgecolor='black')
plt.title('Count of Ratings per Author')
plt.xlabel('Number of Ratings')
plt.ylabel('Count of Authors')
plt.show()


## Simple sentiment analysis

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

* We used the `TextBlob` library to perform a simple sentiment analysis on the review text.
* First, we dropped reviews with no text, as they would not provide any information for sentiment analysis.
* We then calculated the polarity of each review, which ranges from -1 (most negative) to 1 (most positive).
* Plotting a histogram and summary statistics, we see that most reviews are moderately positive with a mean polarity of 0.36 and a standard deviation of 0.31. This is consistent with our earlier observation that most ratings are positive (5 stars).
* The distribution is roughly bell-shaped, but with a density spike at 0 and a few more spikes above 0.5. 


In [ ]:
# Simple sentiment analysis on review text

# Add a column to the dataframe with the sentiment of the review
df_dropped = df.dropna(subset=['text'])
df_dropped['sentiment'] = df_dropped['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# print summary statistics
print("Summary statistics for sentiment:")
print(df_dropped['sentiment'].describe())

# plot the distribution of sentiment
plt.hist(df_dropped['sentiment'], bins=100, edgecolor='black')
plt.title('Distribution of Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

There is a moderate positive correlation between sentiment score and rating, with a correlation coefficient of 0.547. This suggests that reviews with higher ratings tend to have more positive sentiment scores, which is expected.

We were worried that meaningful sentiment might not be extracted from short reviews, but the correlation suggests that the sentiment analysis is capturing the sentiment of the reviews decently well.

In [ ]:
# Find correlation between sentiment and rating
correlation_matrix = df_dropped[['rating', 'sentiment']].corr()
print("Correlation matrix:")
print(correlation_matrix)

## Summary of findings

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

We see that the majority of ratings given by Google users are 5 out of 5. This aligns with the Google text review sentiment analysis, which found that the majority of the reviews had a postive sentiment. We also found that there is a weak correlation between the length of a text review and the rating given by the same Google user. However, it could be helpful to note that based on the visualization displayed above, the review length was longest for either 1 or 5 star reviews. In the sentiment review, we found that majority of the text reviews result in a sentiment score near 0, which suggests that many reviews are neutral. The second most common sentiment scores are those near 1, which suggests that there are also many (but not as many) reviews that are highly positive. 


## Revised Project Question

<div style="background-color:#3F7FBF; color:white; padding:10px"> 

Create a restaurant recommender based on text reviews and a user's favorite restaurants. 
